# <font color='blue'>Vamos a generar el ndvi de cada escena entre 2007 y 2010 para poder ejecutar el análisis de cobertura de usos del suelo. Lo que necesitamos realmente es discriminar suelo desnudo (ligado a obras) de resto de coberturas</font>

El primer paso es listar todas las escenas de la fecha de interes 

In [1]:
%matplotlib inline 

import os
import rasterio
import subprocess
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path_nor = r'Z:\Proyectos\marina_valdecanas\capas\teledeteccion\Landsat\nor'
path_ori = r'Z:\Proyectos\marina_valdecanas\capas\teledeteccion\Landsat\ori'

years = [i for i in range(2007, 2011)]

In [3]:
years

[2007, 2008, 2009, 2010]

In [7]:
scenes_ori = [os.path.join(path_ori, sc) for sc in os.listdir(path_ori) if int(sc[:4]) in years]
scenes_nor = [os.path.join(path_nor, sc) for sc in os.listdir(path_nor) if int(sc[:4]) in years]

print(scenes_ori[:2])
print(len(scenes_nor))

['Z:\\Proyectos\\marina_valdecanas\\capas\\teledeteccion\\Landsat\\ori\\20100926le7202_32', 'Z:\\Proyectos\\marina_valdecanas\\capas\\teledeteccion\\Landsat\\ori\\20080718le7202_32']
48


Ya tenemos la lista de escenas que nos interesan

In [5]:
def ndvi(nir, red):
    
    '''funcion para calcular el ndvi'''
    
    ndvipath = r'I:\ndvi_PIR\ndvis'
    escena = os.path.split(os.path.split(nir)[0])[1]
    outfile = os.path.join(ndvipath, escena + '__ndvi__.TIF')
    
    with rasterio.open(nir) as _nir:
        NIR = _nir.read()
        #NIR = np.where(NIR==0, -9999, NIR)
        NIRM = np.ma.masked_array(NIR, mask=(NIR == 0))
    
    with rasterio.open(red) as _red:
        RED = _red.read()
        REDM = np.ma.masked_array(RED, mask=(RED == 0))
    
    #Es importante el poner el *^"*· astype(float) porque a pesar de ser float, de no usarlo los valores negativos se van a positivos
    ndvi = np.true_divide((NIRM.astype(float)-REDM.astype(float)), (NIRM+REDM))
    ndvi[ndvi<-1] = -9999
    ndvi[ndvi>1] = -9999

    profile = _nir.meta
    profile.update(dtype=rasterio.float32)
    profile.update(nodata=-9999)

    with rasterio.open(outfile, 'w', **profile) as dst:
        dst.write(ndvi.astype(rasterio.float32))
        print('ndvi', outfile, 'realizado')
        

En principio tenemos la función que cálcula el ndvi, ahora vamos a aplicar la función a las Landsat 7 y 5 de nuestro directorio.
No necesitamos Landsat 8, porque todas las escenas son anteriores a 2013!

In [6]:
for sc in scenes_ori:
    
    bandas = []
    
    for e in os.listdir(sc):
                        
        if e.endswith('B4.TIF'):
            b4 = os.path.join(sc, e)
            #print(b5)
            bandas.append(b4)
        elif e.endswith('B3.TIF'):
            b3 = os.path.join(sc, e)
            #print(b4)
            bandas.append(b3)
            
    if len(bandas) == 2:
        ndvi(b4, b3)

C:\Users\Diego\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Diego\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Diego\Anaconda3\lib\site-packages\rasterio\__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


ndvi I:\ndvi_PIR\ndvis\20100926le7202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20080718le7202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20090729lt5202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20081123le7202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20100411lt5202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20090227le7202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20090315le7202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20100801lt5202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20090923le7202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20071020le7202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20071105le7202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20070622lt5202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20090611lt5202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20070801le7202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20091102lt5202_32__ndvi__.TIF realizado
ndvi I:\ndvi_PIR\ndvis\20090211le7202_32__ndvi__.TIF re

In [ ]:
b_3 = r'I:\ndvi_PIR\20070708l5tm202_32\20070708l5tm202_32_grn1_b3.img'
b_4 = r'I:\ndvi_PIR\20070708l5tm202_32\20070708l5tm202_32_grn1_b4.img'
ndvi = r'I:\ndvi_PIR\ndvis\LE72020322008200EDC00_ndvi.TIF'

NDVI = rasterio.open(ndvi)
NDVIA = NDVI.read()
NDVIA

In [ ]:
b3 = rasterio.open(b_3)
B3 = b3.read()
B3M = np.ma.masked_array(B3, mask=(B3 == 10001))

b4 = rasterio.open(b_4)
B4 = b4.read()
B4M = np.ma.masked_array(B4, mask=(B4 == 10001))

In [ ]:
ndvi = (B3M.astype(float)-B4M.astype(float))/(B3M+B4M)
ndvi[(ndvi > 1)] = -9999


In [ ]:
profile = b3.meta
profile.update(dtype=rasterio.float32)
profile.update(nodata=-9999)

with rasterio.open(outfile, 'w', **profile) as dst:
    dst.write(ndvi.astype(rasterio.float32))

In [ ]:
for k, v in b3.meta.items():
    print(k, v)

In [ ]:
%pylab inline
import matplotlib.pyplot as plt

<font size=5 color='red'><center>Mascara al entorno de la isla</center></font>

In [4]:
ruta = r'I:\ndvi_PIR\ndvis'
shape = r'I:\ndvi_PIR\recorte.shp'
crop = "-crop_to_cutline"

for i in os.listdir(ruta):
    
    if i.endswith('ndvi__.TIF'):
        
        ndvi = os.path.join(ruta, i)
        escena = os.path.split(ndvi)[1][:-4] + '_rec.TIF'
        
        print(escena)
        #usamos Gdalwarp para realizar las mascaras, llamandolo desde el modulo subprocess
        cmd = ["gdalwarp", "-dstnodata" , "-9999" , "-cutline", ]
        path_masks = os.path.join(ruta, 'rec')
        if not os.path.exists(path_masks):
            os.makedirs(path_masks)
            
        salida = os.path.join(path_masks, escena)

        cmd.insert(4, shape)
        cmd.insert(5, crop)
        cmd.insert(6, ndvi)
        cmd.insert(7, salida)

        proc = subprocess.Popen(cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        stdout,stderr=proc.communicate()
        exit_code=proc.wait()

        if exit_code: 
            raise RuntimeError(stderr)

20070310le7202_32__ndvi___rec.TIF
20070529le7202_32__ndvi___rec.TIF
20070622lt5202_32__ndvi___rec.TIF
20070708lt5202_32__ndvi___rec.TIF
20070724lt5202_32__ndvi___rec.TIF
20070801le7202_32__ndvi___rec.TIF
20070809lt5202_32__ndvi___rec.TIF
20070910lt5202_32__ndvi___rec.TIF
20070926lt5202_32__ndvi___rec.TIF
20071020le7202_32__ndvi___rec.TIF
20071105le7202_32__ndvi___rec.TIF
20080209le7202_32__ndvi___rec.TIF
20080718le7202_32__ndvi___rec.TIF
20080803le7202_32__ndvi___rec.TIF
20081123le7202_32__ndvi___rec.TIF
20081225le7202_32__ndvi___rec.TIF
20090211le7202_32__ndvi___rec.TIF
20090227le7202_32__ndvi___rec.TIF
20090315le7202_32__ndvi___rec.TIF
20090502le7202_32__ndvi___rec.TIF
20090518le7202_32__ndvi___rec.TIF
20090611lt5202_32__ndvi___rec.TIF
20090627lt5202_32__ndvi___rec.TIF
20090713lt5202_32__ndvi___rec.TIF
20090729lt5202_32__ndvi___rec.TIF
20090830lt5202_32__ndvi___rec.TIF
20090907le7202_32__ndvi___rec.TIF
20090915lt5202_32__ndvi___rec.TIF
20090923le7202_32__ndvi___rec.TIF
20091001lt5202

In [ ]:
os.getcwd()

In [ ]:
a = "Z:\\Proyectos\\marina_valdecanas\\capas\\teledeteccion\\Landsat\\ori\\20120119le7202_32\\L7202322008200"
b = os.path.split(os.path.split(a)[0])[1]
b

In [9]:
import os

path = r'I:\ndvi_PIR\ndvis'
c=0
for i in os.listdir(path):
    
    if os.path.isdir(os.path.join(path, i)):
        
        for banda in os.listdir(os.path.join(path, i)):
            
            if banda.endswith('ndvi2.TIF'):
                c+=1
                print(i, c)            
            
            
            
            

20070622l5tm202_32 1
20070708l5tm202_32 2
20070724l5tm202_32 3
20070809l5tm202_32 4
20070910l5tm202_32 5
20070926l5tm202_32 6
20090611l5tm202_32 7
20090627l5tm202_32 8
20090713l5tm202_32 9
20090729l5tm202_32 10
20090830l5tm202_32 11
20090915l5tm202_32 12
20091017l5tm202_32 13
20100206l5tm202_32 14
20100310l5tm202_32 15
20100411l5tm202_32 16
20100630l5tm202_32 17
20100716l5tm202_32 18
20100801l5tm202_32 19
20101105l5tm202_32 20
20110124l5tm202_32 21
20110225l5tm202_32 22
20110516l5tm202_32 23
20110601l5tm202_32 24
20110804l5tm202_32 25
20110905l5tm202_32 26


In [10]:
from shutil import copyfile

outpath = r'I:\ndvi_PIR\ndvis\ndvis'
path = r'I:\ndvi_PIR\ndvis'

for i in os.listdir(path):
    
    if os.path.isdir(os.path.join(path, i)):
        
        for banda in os.listdir(os.path.join(path, i)):
            
            if 'ndvi2' in banda:
                
                ndvi = os.path.join(os.path.join(path, i), banda)
                outndvi = os.path.join(outpath, banda)
                copyfile(ndvi, outndvi)


SameFileError: 'I:\\ndvi_PIR\\ndvis\\ndvis\\20070622l5tm202_32_ndvi2.tfw' and 'I:\\ndvi_PIR\\ndvis\\ndvis\\20070622l5tm202_32_ndvi2.tfw' are the same file